## Alexnet implementation in Tensorflow
source: https://github.com/allenai/XNOR-Net

In [1]:
import numpy as np
from scipy import misc

class Dataset():
    def __init__(self, paths, labels, train, test, valid, batch, seed=777):
        l = Dataset.one_hot(labels)
        self.train = Dataset.preprocess([paths[i] for i in train]), np.array([l[i] for i in train])
        self.test = Dataset.preprocess([paths[i] for i in test], True), np.array([l[i] for i in test])
        self.valid = Dataset.preprocess([paths[i] for i in valid], True), np.array([l[i] for i in valid])
        self.train_next = 0
        self.batch = batch
        self.starting = self.train
        self.seed = seed
        np.random.seed(seed)
        
    def one_hot(labels):
        size = len(labels)
        max_value = max(labels)
        one_hot = np.zeros((size, max_value+1))
        for i in np.arange(size):
            one_hot[i][labels[i]] = 1
        return one_hot
    
    def preprocess(paths, centered=False):
        size = len(paths)
        result = []
        for i in range(size):
            foto = misc.imread(paths[i])
            scale = 256/(min(foto.shape[:2]))
            foto = misc.imresize(foto, scale)
            if centered:
                x = int(np.floor((foto.shape[0]-224)/2))
                y = int(np.floor((foto.shape[1]-224)/2))            
            else:
                x = np.random.randint(0,foto.shape[0]-224)
                y = np.random.randint(0,foto.shape[1]-224)
            result.append(foto[x:224+x,y:224+y,:])
        return np.array(result)
    
    def next_batch(self):
        assert self.train_next <= len(self.train[0])
        if self.train_next == len(self.train[0]):
            self.train_next = 0
        if self.train_next + self.batch > len(self.train[0]):
            batch =  self.train[0][self.train_next:], self.train[1][self.train_next:]
        else:
            batch = self.train[0][self.train_next:self.train_next + self.batch], self.train[1][self.train_next:self.train_next + self.batch]
        self.train_next += len(batch[0])
        return batch
    
    def vset(self):
        return self.valid
    
    def tset(self):
        return self.test
    
    def shuffle(self):
        self.train_next = 0
        perm = np.random.permutation(len(self.train[0]))
        self.train = self.train[0][perm], self.train[1][perm]
    
    def reset(self):
        self.train = self.starting
        np.random.seed(self.seed)
        self.shuffle()

In [1]:
from scipy.io import loadmat
import glob

ids = loadmat('setid.mat')

test = ids['trnid'][0] -1
train = ids['tstid'][0] -1
valid = ids['valid'][0] - 1

labels = loadmat('imagelabels.mat')['labels'][0]
labels -= 1
files = sorted(glob.glob('jpg/*.jpg'))

In [ ]:
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]
        
data = Dataset(files, labels, train, test, valid, 50)
validation = list(chunks(data.vset()[0], 50)), list(chunks(data.vset()[1], 50))
test = list(chunks(data.tset()[0], 50)), list(chunks(data.tset()[1], 50))

In [ ]:
import tensorflow as tf
import numpy as np

def weight_variable(shape, name=None):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial, name=name)

def bias_variable(shape, name=None):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial, name=name)

lr = tf.placeholder(tf.float32)
x = tf.placeholder(tf.float32, [None, 224*224*3])
x_image = tf.reshape(x, [-1,224,224,3])

def conv2d(x, W, b, strides=[1,1,1,1], padding=[[0,0],[0,0],[0,0],[0,0]]):
    x_padded = tf.pad(x, padding, 'CONSTANT')
    h_conv = tf.nn.conv2d(x_padded, W, strides, 'VALID') + b
    normed = tf.contrib.layers.batch_norm(h_conv, epsilon=1e-3)
    return tf.nn.relu(normed)

def max_pool(x, ksize, strides):
    return tf.nn.max_pool(x, ksize=ksize, strides=strides, padding='VALID')

with tf.device("/gpu:0"):
    W_conv1 = weight_variable([11, 11, 3, 96])
    b_conv1 = bias_variable([96])
    h_conv1 = conv2d(x_image, W_conv1, b_conv1, [1, 4, 4, 1], [[0,0],[2,2],[2,2],[0,0]])
    h_pool1 = max_pool(h_conv1, [1,3,3,1], [1,2,2,1])

    W_conv2 = weight_variable([5, 5, 96, 256])
    b_conv2 = bias_variable([256])
    h_conv2 = conv2d(h_pool1, W_conv2, b_conv2, [1, 1, 1, 1], [[0,0],[2,2],[2,2],[0,0]])
    h_pool2 = max_pool(h_conv2, [1,3,3,1], [1,2,2,1])

    W_conv3 = weight_variable([3, 3, 256, 384])
    b_conv3 = bias_variable([384])
    h_conv3 = conv2d(h_pool2, W_conv3, b_conv3, [1, 1, 1, 1], [[0,0],[1,1],[1,1],[0,0]])

    W_conv4 = weight_variable([3, 3, 384, 384])
    b_conv4 = bias_variable([384])
    h_conv4 = conv2d(h_conv3, W_conv4, b_conv4, [1, 1, 1, 1], [[0,0],[1,1],[1,1],[0,0]])

    W_conv5 = weight_variable([3, 3, 384, 256])
    b_conv5 = bias_variable([256])
    h_conv5 = conv2d(h_conv4, W_conv5, b_conv5, [1, 1, 1, 1], [[0,0],[1,1],[1,1],[0,0]])
    h_pool3 = max_pool(h_conv5, [1,3,3,1], [1,2,2,1])

    prob = tf.placeholder(tf.float32)
    h_drop1 = tf.nn.dropout(h_pool3, prob)

    W_conv6 = weight_variable([6, 6, 256, 4096])
    b_conv6 = bias_variable([4096])
    h_conv6 = conv2d(h_drop1, W_conv6, b_conv6, [1, 1, 1, 1])

    h_drop2 = tf.nn.dropout(h_conv6, prob)

    W_conv7 = weight_variable([1, 1, 4096, 4096])
    b_conv7 = bias_variable([4096])
    h_conv7 = conv2d(h_drop2, W_conv7, b_conv7)

    nClasses = 102
    W_conv8 = weight_variable([1, 1, 4096, nClasses])
    b_conv8 = bias_variable([nClasses])
    y = conv2d(h_conv7, W_conv8, b_conv8)[:,0,0,:]

    y_ = tf.placeholder(tf.float32, [None, nClasses])

    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y, labels=y_))
    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    train_step = tf.train.AdamOptimizer(lr).minimize(cross_entropy)

saver = tf.train.Saver()

In [ ]:
from datetime import datetime as time

def debug(text, start):
    print(text + " Time spent: " + str(time.now()-start).split(".")[0])

maxim = 0
sess = tf.Session()
sess.as_default()
rate = np.float(0.001)
accuracies = []
sess.run(tf.global_variables_initializer())
start = time.now()
data.reset()
for k in range(420):
    for i in range(123):
        batch = data.next_batch()
        train_step.run(session=sess, feed_dict={x_image: batch[0], y_: batch[1], prob: 0.5, lr: rate})
        print(len(batch[0]))
    if k % 10 == 0:
        val = []
        for i in range(len(validation[0])):
            val.append(accuracy.eval(session=sess, feed_dict={x_image: validation[0][i], y_: validation[1][i], prob: 1}))
        val = np.mean(val)
        accuracies.append(val)
        if maxim <= val:
            maxim = val
            save_path = saver.save(sess, "model" + ".ckpt")    
        debug("Learning rate: " + str(rate) + " Epoch: " + str(k+1) + " Accuracy: " + str(val), start)
    if k % 160 == 0 and k != 0:
        rate *= 0.1

